# DS-SF-30 | Unit Project 3: Machine Learning Modeling

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.

In [1]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)
pd.set_option('display.notebook_repr_html', True)

import statsmodels.formula.api as smf

from sklearn import linear_model

In [2]:
df = pd.read_csv(os.path.join('..', '..', 'dataset', 'dataset-ucla-admissions.csv'))
df.dropna(inplace = True)

df

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0
...,...,...,...,...
395,0,620.0,4.00,2.0
396,0,560.0,3.04,3.0
397,0,460.0,2.63,2.0
398,0,700.0,3.65,2.0


## Part A.  Frequency Table

> ### Question 1.  Create a frequency table for `prestige` and whether an applicant was admitted.

In [3]:
# TODO
pd.crosstab(df.admit, df.prestige)

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


## Part B.  Variable Transformations

> ### Question 2.  Create a one-hot encoding for `prestige`.

In [4]:
# TODO
Pres_df = pd.get_dummies(df.prestige, prefix = 'Prestige')

Pres_df.columns = ['Prestige_1', 'Prestige_2', 'Prestige_3','Prestige_4']

Pres_df

,Prestige_1,Prestige_2,Prestige_3,Prestige_4
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0
...,...,...,...,...
395,0.0,1.0,0.0,0.0
396,0.0,0.0,1.0,0.0
397,0.0,1.0,0.0,0.0
398,0.0,1.0,0.0,0.0


> ### Question 3.  How many of these binary variables do we need for modeling?

Answer: 3

> ### Question 4.  Why are we doing this?

Answer: Because when you know 3 out of 4 variables, you can derive the remainder since the sum of all will always be 1. If you have all variables included, we now have multicollinearity issues. 

> ### Question 5.  Add all these binary variables in the dataset and remove the now redundant `prestige` feature.

In [5]:
# TODO

df = df.join([Pres_df])
df.drop('prestige', axis = 1, inplace = True)
df

,admit,gre,gpa,Prestige_1,Prestige_2,Prestige_3,Prestige_4
0,0,380.0,3.61,0.0,0.0,1.0,0.0
1,1,660.0,3.67,0.0,0.0,1.0,0.0
2,1,800.0,4.00,1.0,0.0,0.0,0.0
3,1,640.0,3.19,0.0,0.0,0.0,1.0
4,0,520.0,2.93,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
395,0,620.0,4.00,0.0,1.0,0.0,0.0
396,0,560.0,3.04,0.0,0.0,1.0,0.0
397,0,460.0,2.63,0.0,1.0,0.0,0.0
398,0,700.0,3.65,0.0,1.0,0.0,0.0


## Part C.  Hand calculating odds ratios

Let's develop our intuition about expected outcomes by hand calculating odds ratios.

> ### Question 6.  Create a frequency table for `prestige = 1` and whether an applicant was admitted.

In [6]:
# TODO
pd.crosstab(df.admit, df.Prestige_1)

Prestige_1,0.0,1.0
admit,,
0,243,28
1,93,33


> ### Question 7.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the most prestigious undergraduate schools.

In [ ]:
# TODO
Odd of 0.083

> ### Question 8.  Now calculate the odds of admission for undergraduates who did not attend a #1 ranked college.

In [ ]:
# TODO
Odd of 0.234

> ### Question 9.  Finally, what's the odds ratio?

In [ ]:
# TODO
1 to 2.82

> ### Question 10.  Write this finding in a sentence.

Answer: for every student admitted from top undergraduate program, 2.82 students were admitted from not top undergraduate programs. 

> ### Question 11.  Use the frequency table above to calculate the odds of being admitted to graduate school for applicants that attended the least prestigious undergraduate schools.  Then calculate their odds ratio of being admitted to UCLA.  Finally, write this finding in a sentence.

In [7]:
# TODO
pd.crosstab(df.admit, df.Prestige_4)


Prestige_4,0.0,1.0
admit,,
0,216,55
1,114,12


Answer: Odd of 0.03
Students from the least prestigious schools have 3% chance of acceptance. 

## Part C. Analysis using `statsmodels`

> ### Question 12.  Fit a logistic regression model predicting admission into UCLA using `gre`, `gpa`, and the `prestige` of the undergraduate schools.  Use the highest prestige undergraduate schools as your reference point.

In [8]:
# TODO

X = df[ ['gre','gpa','Prestige_1','Prestige_2','Prestige_3']]
model_Prestige = linear_model.LogisticRegression().\
    fit(X, Pres_df.Prestige_4)

    

> ### Question 13.  Print the model's summary results.

In [9]:
# TODO
print model_Prestige.coef_
print model_Prestige.intercept_

[[ -3.25798947e-05   2.30901643e-01  -4.00555794e+00  -4.72038854e+00
   -4.59774553e+00]]
[ 0.57555829]


> ### Question 14.  What are the odds ratios of the different features and their 95% confidence intervals?

In [16]:
# TODO
Train_df = df.columns[1:]
logit = smf.Logit(df['admit'], df[Train_df])
result = logit.fit()

result.summary()
print np.exp(result.params)

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6
gre           1.002221
gpa           2.180027
Prestige_1    0.020716
Prestige_2    0.010494
Prestige_3    0.005432
Prestige_4    0.004382
dtype: float64


> ### Question 15.  Interpret the odds ratio for `prestige = 2`.

Answer: students from prestige 2 schools have approx 50% chance to admission compared to prestige 1 school's students

> ### Question 16.  Interpret the odds ratio of `gpa`.

Answer:

> ### Question 17.  Assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [ ]:
# TODO

Answer:

## Part D. Moving the model from `statsmodels` to `sklearn`

> ### Question 18.  Let's assume we are satisfied with our model.  Remodel it (same features) using `sklearn`.  When creating the logistic regression model with `LogisticRegression(C = 10 ** 2)`.

In [ ]:
# TODO

> ### Question 19.  What are the odds ratios for the different variables and how do they compare with the odds ratios calculated with `statsmodels`?

In [ ]:
# TODO

Answer:

> ### Question 20.  Again, assuming a student with a GRE of 800 and a GPA of 4.  What is his/her probability of admission  if he/she come from a tier-1, tier-2, tier-3, or tier-4 undergraduate school?

In [ ]:
# TODO

Answer: